## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# model=keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor,pooling=None, classes=2)

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                  input_tensor=input_tensor,
                                  pooling=None, classes=10)
model.summary()


94773248/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
__________________________

## 添加層數

In [5]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 179


## 鎖定特定幾層不要更新權重

In [6]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 7s 0us/step
(50000, 32, 32, 3)


## Training

In [9]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=10)

Epoch 1/10
1563/1563 [==============================] - 44s 28ms/step - loss: 1.4310 - accuracy: 0.4990
Epoch 2/10
1563/1563 [==============================] - 45s 28ms/step - loss: 1.3596 - accuracy: 0.5221
Epoch 3/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.3033 - accuracy: 0.5393
Epoch 4/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.2230 - accuracy: 0.5698
Epoch 5/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.1814 - accuracy: 0.5818
Epoch 6/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.1317 - accuracy: 0.5997
Epoch 7/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.1363 - accuracy: 0.5982
Epoch 8/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0838 - accuracy: 0.6163
Epoch 9/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0324 - accuracy: 0.6351
Epoch 10/10
1563/1563 [==============================] - 45s 29m